# Solar Radiation Stations Catalog

Ground measured solar irradiance data is extremely valuable and critical for benchmarking solar radiation products and understanding long-term trends in the radiation budget. However, due to high costs and maintenance requirements, there are only a few hundred high-quality stations globally. Partly due to the scarcity, it has historically been difficult to determine if and where there is a solar radiation meonitoring stations. This catalog provides an overview of available solar radiation monitoring stations worldwide and supporting metadata.

The catalog is currently in the draft phase, where only the BSRN stations are included as a demonstration.

A list of stations can be found [here](../station_listing) and downloaded as a csv file [here](https://github.com/AssessingSolar/radiationstations/blob/main/radiationstations.csv).

## Overview of stations

To find the nearest station to a point of interest, check out the interactive map below. It's possible to click on a given station to get the station name.

In [1]:
import pandas as pd
import folium

df = pd.read_csv('radiationstations.csv', sep=';', encoding='latin1')

# Initialize Folium map
m = folium.Map(
    location=[0, 15],
    zoom_start=0,
    min_zoom=0,
    max_bounds=True,
    tiles='openstreetmap',
    )

# Add each station to the map
for index, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['Station full name']} ({row.name})",
        icon=folium.Icon(color='blue')
    ).add_to(m)

m  # Show the map

## Future improvements
Some of the future improvements include coloring of the stations according to the network (e.g., BSRN, NREL, SRML, Enermena). It can then be possible to toggle the visibility of the different stations networks shown on the map.

Also, the table of stations above will be made sortable and with clickable url's.